# YOLOv5 Person Detection - COCO Dataset (1/10 샘플)

This notebook trains YOLOv5 person detection model using 1/10 of COCO dataset.

**Note**: Uses inference-based evaluation instead of validation loss calculation.

## 1. Setup and Library Import

## 📦 Import Libraries


In [1]:
import argparse
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import yaml
from pathlib import Path

from utils.dataloaders import create_dataloader
from utils.general import check_dataset
from utils.torch_utils import select_device
from models.yolo import Model
from utils.loss import ComputeLoss

print("✓ Libraries imported")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

✓ 라이브러리 임포트 완료


## Dataset Preprocessing Function



In [ ]:
# Filter images with targets only
def filter_images_with_labels(img_dir, label_dir):
    """Return image paths with targets only"""
    img_dir = Path(img_dir)
    label_dir = Path(label_dir)
    
    valid_images = []
    all_images = sorted(img_dir.glob('*.jpg'))
    
    print(f"Filtering images...")
    for img_path in tqdm(all_images, desc='Checking labels'):
        # Label file path
        label_path = label_dir / f"{img_path.stem}.txt"
        
        # Include if label file exists and not empty
        if label_path.exists():
            with open(label_path, 'r') as f:
                content = f.read().strip()
                if content:  # If has content
                    valid_images.append(str(img_path))
    
    return valid_images

# Train: Filter images with targets only
train_img_dir = '../datasets/coco/images/train2017'
train_label_dir = '../datasets/coco/labels/train2017'

print("\n[Train] Filtering images with person...")
train_images = filter_images_with_labels(train_img_dir, train_label_dir)
total_train_imgs = len(list(Path(train_img_dir).glob('*.jpg')))
print(f"  Total images: {total_train_imgs}")
print(f"  With person: {len(train_images)}")
print(f"  Filter ratio: {100 * len(train_images) / total_train_imgs:.1f}%")

# Val: Filter images that contain the target
val_img_dir = '../datasets/coco/images/val2017'
val_label_dir = '../datasets/coco/labels/val2017'

print("\n[Val] Filtering images with person...")
val_images = filter_images_with_labels(val_img_dir, val_label_dir)
total_val_imgs = len(list(Path(val_img_dir).glob('*.jpg')))
print(f"  Total images: {total_val_imgs}")
print(f"  With person: {len(val_images)}")
print(f"  Filter ratio: {100 * len(val_images) / total_val_imgs:.1f}%")

# Save filtered image list to text file
train_list_file = '../datasets/coco/train2017_with_person.txt'
val_list_file = '../datasets/coco/val2017_with_person.txt'

with open(train_list_file, 'w') as f:
    f.write('\n'.join(train_images))

with open(val_list_file, 'w') as f:
    f.write('\n'.join(val_images))

print(f"\n✓ Image list saved")

# Train dataloader - use filtered images only
train_loader_full, train_dataset = create_dataloader(
    path=train_list_file,  # pass as txt file
    imgsz=640,
    batch_size=args.batch_size,
    stride=32,
    single_cls=True,
    hyp=None,
    augment=False,
    cache=False,
    rect=False,
    rank=-1,
    workers=args.num_workers,
    shuffle=True,
    prefix='train: '
)

# Val dataloader - use filtered images only
val_loader_full, val_dataset = create_dataloader(
    path=val_list_file,  # pass as txt file
    imgsz=640,
    batch_size=args.batch_size,
    stride=32,
    single_cls=True,
    hyp=None,
    augment=False,
    cache=False,
    rect=False,
    rank=-1,
    workers=args.num_workers,
    shuffle=False,
    prefix='val: '
)

# 1/10 sampling (from full data)
train_sampled_size = len(train_dataset) // 10
val_sampled_size = len(val_dataset) // 10

train_indices = list(range(0, len(train_dataset), 10))  # 1 out of every 10
val_indices = list(range(0, len(val_dataset), 10))

from torch.utils.data import Subset
train_subset = Subset(train_dataset, train_indices)
val_subset = Subset(val_dataset, val_indices)

train_loader = torch.utils.data.DataLoader(
    train_subset,
    batch_size=args.batch_size,
    shuffle=True,
    num_workers=args.num_workers,
    collate_fn=train_dataset.collate_fn,
    pin_memory=True
)

val_loader = torch.utils.data.DataLoader(
    val_subset,
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=args.num_workers,
    collate_fn=val_dataset.collate_fn,
    pin_memory=True
)

print(f"\nDataloader ready")
print(f"  Train total: {len(train_dataset)} → Sampled: {len(train_subset)} (1/10)")
print(f"  Val total: {len(val_dataset)} → Sampled: {len(val_subset)} (1/10)")

## Baseline Model Training

In [8]:
!python train.py --img 640 --batch 16 --epoch 10 --data data/person_final.yaml --weights yolov5s.pt --workers 4 --name people_detect_run 

train: weights=yolov5s.pt, cfg=, data=data/person_final.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=4, project=runs/train, name=people_detect_run, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: ⚠️ YOLOv5 is out of date by 2 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-448-gdeec5e45 Python-3.9.23 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24090MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay

## Apply Pruning

In [ ]:
import torch.nn as nn
SENSITIVITY = 0.7
 
def prune_by_std(model, s):

    pruned_count = 0
    total_count = 0
    
    print(f"\nPruning with sensitivity={s}")
    print("="*70)
    
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            weight = module.weight.data
            std = weight.std().item()
            threshold = std * s
            mask = weight.abs() < threshold
            weight[mask] = 0.0
            
            layer_pruned = mask.sum().item()
            layer_total = weight.numel()
            pruned_count += layer_pruned
            total_count += layer_total
            
            ratio = 100.0 * layer_pruned / layer_total
            print(f"{name:<45} | {ratio:>6.2f}% | {layer_pruned:>7,}/{layer_total:>7,}")
    
    overall = 100.0 * pruned_count / total_count
    print("="*70)
    print(f"{'TOTAL':<45} | {overall:>6.2f}% | {pruned_count:>7,}/{total_count:>7,}")
    print("="*70)
    
    return pruned_count, total_count

print("\n" + "="*60)
print(" PRUNING ")
print("="*60)

ckpt = torch.load('runs/train/exp2/weights/best.pt', map_location = device, weights_only=False)
best_model = ckpt['model'].float().to(device)

# perform Pruning 
pruned, total = prune_by_std(best_model, SENSITIVITY)

save_ckpt = {
    'model': best_model,  
    'epoch': ckpt.get('epoch', -1),
    'best_fitness': ckpt.get('best_fitness', None),
    'optimizer': None,  
    'date': 'Pruned Model Checkpoint',
    'ema': None,  
}

# torch.save(save_ckpt, 'runs/pruned/best_pruned.pt')
torch.save(save_ckpt, 'runs/pruned/best_pruned_70.pt')



 PRUNING 

Pruning with sensitivity=0.7
model.0.conv                                  |  70.37% |   2,432/  3,456
model.1.conv                                  |  71.10% |  13,106/ 18,432
model.2.cv1.conv                              |  73.29% |   1,501/  2,048
model.2.cv2.conv                              |  70.26% |   1,439/  2,048
model.2.cv3.conv                              |  69.92% |   2,864/  4,096
model.2.m.0.cv1.conv                          |  69.43% |     711/  1,024
model.2.m.0.cv2.conv                          |  71.27% |   6,568/  9,216
model.3.conv                                  |  61.47% |  45,323/ 73,728
model.4.cv1.conv                              |  58.42% |   4,786/  8,192
model.4.cv2.conv                              |  64.84% |   5,312/  8,192
model.4.cv3.conv                              |  59.55% |   9,757/ 16,384
model.4.m.0.cv1.conv                          |  52.93% |   2,168/  4,096
model.4.m.0.cv2.conv                          |  58.38% |  21,523/ 36,8

## Validate Pruned Model

In [8]:
!python val.py --weights runs/pruned/best_pruned_70.pt --data data/person_final.yaml --img 640

val: data=data/person_final.yaml, weights=['runs/pruned/best_pruned_70.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-448-gdeec5e45 Python-3.9.23 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24090MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /home/minhyuk/Yolov5_nano/datasets/coco/person_only_val2017.cache.
                 Class     Images  Instances          P          R      mAP50   
                   all       2693      10777      0.109      0.322      0.141     0.0625
Speed: 0.1ms pre-process, 1.1ms inference, 0.3ms NMS per image at shape (32, 3, 640, 640)
Results saved to runs/val/exp6


## Fine-tune Pruned Model

In [18]:
!python train_fine_tuning.py --weights runs/pruned/best_pruned_70.pt --cfg models/yolov5s.yaml --data data/person_final.yaml --epochs 5 --batch-size 64 --hyp data/hyps/hyp.scratch-low.yaml --workers 8 --name pruned_finetuned

train_fine_tuning: weights=runs/pruned/best_pruned_70.pt, cfg=models/yolov5s.yaml, data=data/person_final.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=pruned_finetuned, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: ⚠️ YOLOv5 is out of date by 2 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-448-gdeec5e45 Python-3.9.23 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24090MiB)

hyperparameters: 

# Apply Quantization

In [20]:
import torch
import numpy as np
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix, csc_matrix
import copy

def apply_weight_sharing(model, bits):
    for name, module in model.named_modules():
        if isinstance(module, (torch.nn.Conv2d, torch.nn.Linear)):
            
            dev = module.weight.device
            weight = module.weight.data.cpu().numpy()

            original_shape = weight.shape
            flat_weight = weight.reshape(-1,1)

            min_ = min(flat_weight)
            max_ = max(flat_weight)
            n_clusters = 2**bits
            init_points = np.linspace(min_, max_, num=n_clusters).reshape(-1,1)
            kmeans = KMeans(
                n_clusters=n_clusters,
                init=init_points,
                n_init=1,
                algorithm="lloyd",
                max_iter=10,
                random_state=42
            ).fit(flat_weight)

            new_weight = kmeans.cluster_centers_[kmeans.labels_].reshape(original_shape)

            module.weight.data = torch.from_numpy(new_weight).to(dev)
            print(f"[✓] {name} quantized to {n_clusters} levels ({bits}-bit)")

    print("✅ Weight sharing applied to YOLOv5 successfully!")
    return model
            
ckpt = torch.load('runs/train/pruned_finetuned13/weights/best.pt', map_location=device, weights_only=False)
pruned_model = ckpt['model'].float().to(device)
quantized_model = copy.deepcopy(pruned_model)
quantized_model = apply_weight_sharing(quantized_model, 5)
torch.save({'model':quantized_model}, 'runs/pruned/quantized_best_70.pt')

[✓] model.0.conv quantized to 32 levels (5-bit)
[✓] model.1.conv quantized to 32 levels (5-bit)
[✓] model.2.cv1.conv quantized to 32 levels (5-bit)
[✓] model.2.cv2.conv quantized to 32 levels (5-bit)
[✓] model.2.cv3.conv quantized to 32 levels (5-bit)
[✓] model.2.m.0.cv1.conv quantized to 32 levels (5-bit)
[✓] model.2.m.0.cv2.conv quantized to 32 levels (5-bit)
[✓] model.3.conv quantized to 32 levels (5-bit)
[✓] model.4.cv1.conv quantized to 32 levels (5-bit)
[✓] model.4.cv2.conv quantized to 32 levels (5-bit)
[✓] model.4.cv3.conv quantized to 32 levels (5-bit)
[✓] model.4.m.0.cv1.conv quantized to 32 levels (5-bit)
[✓] model.4.m.0.cv2.conv quantized to 32 levels (5-bit)
[✓] model.4.m.1.cv1.conv quantized to 32 levels (5-bit)
[✓] model.4.m.1.cv2.conv quantized to 32 levels (5-bit)
[✓] model.5.conv quantized to 32 levels (5-bit)
[✓] model.6.cv1.conv quantized to 32 levels (5-bit)
[✓] model.6.cv2.conv quantized to 32 levels (5-bit)
[✓] model.6.cv3.conv quantized to 32 levels (5-bit)
[✓] 

## Weight Sharing 


In [ ]:
import torch
import numpy as np
from sklearn.cluster import KMeans
import copy

def apply_weight_sharing_preserve_zero(model, bits):
    """
    Weight quantization preserving pruning
    - Keep zero weights as zero
    - Cluster only non-zero weights
    """
    for name, module in model.named_modules():
        if isinstance(module, (torch.nn.Conv2d, torch.nn.Linear)):
            
            dev = module.weight.device
            weight = module.weight.data.cpu().numpy()
            original_shape = weight.shape
            
            # Select only non-zero weights
            non_zero_mask = (weight != 0)
            non_zero_weights = weight[non_zero_mask]
            
            if len(non_zero_weights) == 0:
                print(f"[SKIP] {name} - All weights are zero")
                continue
            
            # Cluster only non-zero weights
            flat_weight = non_zero_weights.reshape(-1, 1)
            
            min_ = np.min(flat_weight)
            max_ = np.max(flat_weight)
            n_clusters = 2**bits
            init_points = np.linspace(min_, max_, num=n_clusters).reshape(-1, 1)
            
            kmeans = KMeans(
                n_clusters=n_clusters,
                init=init_points,
                n_init=1,
                algorithm="lloyd",
                max_iter=10,
                random_state=42
            ).fit(flat_weight)
            
            # Replace with quantized values
            quantized_non_zero = kmeans.cluster_centers_[kmeans.labels_].flatten()
            
            # Create new weight matrix (keep zeros)
            new_weight = weight.copy()
            new_weight[non_zero_mask] = quantized_non_zero
            
            # Apply to model
            module.weight.data = torch.from_numpy(new_weight).to(dev)
            
            # Check sparsity
            sparsity = (weight.size - len(non_zero_weights)) / weight.size * 100
            print(f"[✓] {name} quantized to {n_clusters} levels ({bits}-bit) | Sparsity: {sparsity:.2f}%")
    
    return model

print("=" * 70)
print("Pruning-aware Quantization")
print("=" * 70)

ckpt = torch.load('runs/train/pruned_finetuned13/weights/best.pt', map_location=device, weights_only=False)
pruned_model = ckpt['model'].float().to(device)

# Check sparsity (양자화 전)
total_params = 0
zero_params = 0
for module in pruned_model.modules():
    if hasattr(module, 'weight') and module.weight is not None:
        weight = module.weight.data.cpu().numpy()
        total_params += weight.size
        zero_params += np.sum(weight == 0)
        
before_sparsity = zero_params / total_params * 100 if total_params > 0 else 0
print(f"\nSparsity before quantization: {before_sparsity:.2f}%\n")

# apply quantization
quantized_model = copy.deepcopy(pruned_model)
quantized_model = apply_weight_sharing_preserve_zero(quantized_model, 5)

# Check sparsity (after quantization)
total_params = 0
zero_params = 0
for module in quantized_model.modules():
    if hasattr(module, 'weight') and module.weight is not None:
        weight = module.weight.data.cpu().numpy()
        total_params += weight.size
        zero_params += np.sum(weight == 0)
        
after_sparsity = zero_params / total_params * 100 if total_params > 0 else 0
print(f"\nSparsity after quantization: {after_sparsity:.2f}%")
print(f"Sparsity change: {before_sparsity:.2f}% → {after_sparsity:.2f}% (diff: {after_sparsity - before_sparsity:+.2f}%)")

# Save
torch.save({'model': quantized_model}, 'runs/pruned/quantized_best_70_preserve_pruning.pt')
print(f"\n✅ Saved: runs/pruned/quantized_best_70_preserve_pruning.pt")
print("=" * 70)

프루닝 보존 양자화 (Pruning-aware Quantization)

양자화 전 Sparsity: 61.46%

[✓] model.0.conv quantized to 32 levels (5-bit) | Sparsity: 70.37%
[✓] model.1.conv quantized to 32 levels (5-bit) | Sparsity: 71.10%
[✓] model.2.cv1.conv quantized to 32 levels (5-bit) | Sparsity: 73.29%
[✓] model.2.cv2.conv quantized to 32 levels (5-bit) | Sparsity: 70.26%
[✓] model.2.cv3.conv quantized to 32 levels (5-bit) | Sparsity: 69.92%
[✓] model.2.m.0.cv1.conv quantized to 32 levels (5-bit) | Sparsity: 69.43%
[✓] model.2.m.0.cv2.conv quantized to 32 levels (5-bit) | Sparsity: 71.27%
[✓] model.3.conv quantized to 32 levels (5-bit) | Sparsity: 61.47%
[✓] model.4.cv1.conv quantized to 32 levels (5-bit) | Sparsity: 58.42%
[✓] model.4.cv2.conv quantized to 32 levels (5-bit) | Sparsity: 64.84%
[✓] model.4.cv3.conv quantized to 32 levels (5-bit) | Sparsity: 59.55%
[✓] model.4.m.0.cv1.conv quantized to 32 levels (5-bit) | Sparsity: 52.93%
[✓] model.4.m.0.cv2.conv quantized to 32 levels (5-bit) | Sparsity: 58.38%
[✓] mode

## Validate Quantized Model

In [11]:
!python val.py --weights runs/pruned/quantized_best_70_preserve_pruning.pt --data data/person_final.yaml --img 640

val: data=data/person_final.yaml, weights=['runs/pruned/quantized_best_70_preserve_pruning.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-448-gdeec5e45 Python-3.9.23 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24090MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /home/minhyuk/Yolov5_nano/datasets/coco/person_only_val2017.cache.
                 Class     Images  Instances          P          R      mAP50   
                   all       2693      10777      0.805      0.652      0.755      0.481
Speed: 0.1ms pre-process, 1.1ms inference, 0.4ms NMS per image at shape (32, 3, 640, 640)
Results saved to runs/val/exp10


## Fine-tune Quantized Model 

In [12]:
!python train_fine_tuning.py --weights runs/pruned/quantized_best_70_preserve_pruning.pt --cfg models/yolov5s.yaml --data data/person_final.yaml --epochs 5 --batch-size 64 --hyp data/hyps/hyp.scratch-low.yaml --workers 8 --name pruned_finetuned

train_fine_tuning: weights=runs/pruned/quantized_best_70_preserve_pruning.pt, cfg=models/yolov5s.yaml, data=data/person_final.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=pruned_finetuned, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: ⚠️ YOLOv5 is out of date by 2 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-448-gdeec5e45 Python-3.9.23 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24090MiB

# Save as CSR Format

In [ ]:
import numpy as np
import pickle
import os

# 1. Load quantized model
ckpt = torch.load('runs/train/pruned_finetuned17/weights/best.pt', map_location=device, weights_only=False)
# ckpt = torch.load('runs/train/exp2/weights/best.pt', map_location=device, weights_only=False)

model = ckpt['model'].float()

# 2. Convert to CSR format
csr_model = {}

for name, module in model.named_modules():
    if isinstance(module, (torch.nn.Conv2d, torch.nn.Linear)):
        weight = module.weight.data.cpu().numpy()
        
        # Save each row (output channel) as CSR
        csr_rows = []
        for row in weight.reshape(weight.shape[0], -1):
            # Index and value of non-zero elements
            nz_idx = np.where(row != 0)[0]
            nz_val = row[nz_idx]
            
            # Relative indexing (Delta Encoding)
            if len(nz_idx) > 0:
                rel_idx = np.diff(nz_idx, prepend=0).astype(np.uint16)
            else:
                rel_idx = np.array([], dtype=np.uint16)
            
            csr_rows.append({'idx': rel_idx, 'val': nz_val.astype(np.float16)})
        
        csr_model[name] = {
            'shape': weight.shape,
            'rows': csr_rows,
            'bias': module.bias.data.cpu().numpy() if module.bias is not None else None
        }

# 3. Save
os.makedirs('runs/compressed', exist_ok=True)
with open('runs/compressed/csr_model.pkl', 'wb') as f:
    pickle.dump(csr_model, f)

# 4. calculate size and compare
original_size = os.path.getsize('runs/train/pruned_finetuned17/weights/best.pt')
compressed_size = os.path.getsize('runs/compressed/csr_model.pkl')

print(f"Size of Original model :   {original_size / 1024**2:.2f} MB")
print(f"Size of Compressed model :   {compressed_size / 1024**2:.2f} MB")
print(f"Compression ratio :      {(1 - compressed_size/original_size)*100:.1f}%")

Size of Original model :   26.99 MB
Size of Compressed model :   10.94 MB
Compression ratio :      59.5%
